<a href="https://colab.research.google.com/github/rezakhanahmadi342341/matching-learning/blob/main/sigmentaion_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tensorflow import keras
import math
from pathlib import Path
import re
from skimage import measure
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
import matplotlib as mpl
import tqdm
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import normalize
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda

In [38]:
from zipfile import ZipFile

with ZipFile('/content/segmintation.zip', 'r') as files:
  files.extractall('/content/r')


In [39]:
images_path = "/content/r/png_images/IMAGES"
masks_path = "/content/r/png_masks/MASKS"

In [40]:

def standardize(x):
    x = np.array(x, dtype='float64')
    x -= np.min(x)
    x /= np.percentile(x, 98)
    x[x > 1] = 1
    return x

def preprocessing(img):
    image = np.array(img)
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    image = np.zeros_like(image)
    image[:,:,0] = gray
    image[:,:,1] = gray
    image[:,:,2] = gray
    image = standardize(image)
    return image

In [41]:
image_paths = []
masks_paths = []

for imagename in os.listdir(images_path):
  image_paths.append(os.path.join(images_path, imagename))

for imagename in os.listdir(masks_path):
  masks_paths.append(os.path.join(masks_path, imagename))

In [42]:
masks_paths.sort()
image_paths.sort()

In [43]:
train_images = []
train_masks = []

In [44]:
for imagepat in tqdm.tqdm(image_paths):
  image = cv2.imread(imagepat)
  image = cv2.resize(image, (256, 256))
  train_images.append(image)

100%|██████████| 1000/1000 [00:30<00:00, 32.76it/s]


In [45]:
for maskpat in tqdm.tqdm(masks_paths):
  image = cv2.imread(maskpat)
  image = cv2.resize(image, (256,256), interpolation= cv2.INTER_NEAREST)
  train_masks.append(image)

100%|██████████| 1000/1000 [00:05<00:00, 195.11it/s]


In [47]:
x = np.array(train_images)
y = np.array(train_masks)

In [51]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=1)

In [67]:
x_train[4].shape

(256, 256, 3)

In [ ]:
base_model = MobileNetV2(input_shape=(256, 256, 3), include_top=False)
base_model.trainable = False

In [8]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Input, Conv2DTranspose, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2

def unet_with_base_model(input_shape=(256, 256, 3), output_channels=3):
    # Input Layer
    inputs = Input(shape=input_shape)

    # Load MobileNetV2 as the base model (encoder) without the top layers
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False  # Freeze the base model layers

    # Extract specific feature maps from MobileNetV2
    feature_layers = [
        'block_1_expand_relu',   # 128x128
        'block_3_expand_relu',   # 64x64
        'block_6_expand_relu',   # 32x32
        'block_13_expand_relu',  # 16x16
        'block_16_project'       # 8x8
    ]

    # Create a model that outputs multiple feature maps
    feature_extraction_model = Model(
        inputs=base_model.input,
        outputs=[base_model.get_layer(name).output for name in feature_layers]
    )

    # Get feature maps
    features = feature_extraction_model(inputs)

    # Encoder feature maps from base model
    c1, c2, c3, c4, c5 = features

    # Decoder (Upsampling)
    u6 = UpSampling2D((2, 2), interpolation='bilinear')(c5)
    u6 = Conv2DTranspose(512, (3, 3), activation='relu', padding='same')(u6)
    u6 = Concatenate()([u6, c4])  # Skip connection

    u7 = UpSampling2D((2, 2), interpolation='bilinear')(u6)
    u7 = Conv2DTranspose(256, (3, 3), activation='relu', padding='same')(u7)
    u7 = Concatenate()([u7, c3])  # Skip connection

    u8 = UpSampling2D((2, 2), interpolation='bilinear')(u7)
    u8 = Conv2DTranspose(128, (3, 3), activation='relu', padding='same')(u8)
    u8 = Concatenate()([u8, c2])  # Skip connection

    u9 = UpSampling2D((2, 2), interpolation='bilinear')(u8)
    u9 = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(u9)
    u9 = Concatenate()([u9, c1])  # Skip connection

    # Output Layer
    outputs = Conv2D(output_channels, (1, 1), activation='softmax')(u9)

    # Create Model
    model = Model(inputs=inputs, outputs=outputs)

    # Print model summary
    model.summary()

    return model

# Create the model
model = unet_with_base_model(input_shape=(256, 256, 3), output_channels=3)

<ipython-input-8-26fe84a722b4>:10: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11            │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_1 (Functional) │ [(None, 128, 128, 96), │      1,841,984 │ input_layer_11[0][0]   │
│                           │ (None, 64, 64, 144),   │                │                        │
│                           │ (None, 32, 32, 192),   │                │                        │
│                           │ (None, 16, 16, 576),   │                │                        │
│                           │ (None, 8, 8, 320)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d_9           │ (None, 16, 16, 320)    │              0 │ functional_1[0][4]     │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_9        │ (None, 16, 16, 512)    │      1,475,072 │ up_sampling2d_9[0][0]  │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_9             │ (None, 16, 16, 1088)   │              0 │ conv2d_transpose_9[0]… │
│ (Concatenate)             │                        │                │ functional_1[0][3]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d_10          │ (None, 32, 32, 1088)   │              0 │ concatenate_9[0][0]    │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_10       │ (None, 32, 32, 256)    │      2,507,008 │ up_sampling2d_10[0][0] │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_10            │ (None, 32, 32, 448)    │              0 │ conv2d_transpose_10[0… │
│ (Concatenate)             │                        │                │ functional_1[0][2]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d_11          │ (None, 64, 64, 448)    │              0 │ concatenate_10[0][0]   │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_11       │ (None, 64, 64, 128)    │        516,224 │ up_sampling2d_11[0][0] │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_11            │ (None, 64, 64, 272)    │              0 │ conv2d_transpose_11[0… │
│ (Concatenate)             │                        │                │ functional_1[0][1]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d_12          │ (None, 128, 128, 272)  │              0 │ concatenate_11[0][0]   │
│ (UpSampling2D)            │                        │                │                        │
├──────────────────────

 Total params: 6,497,507 (24.79 MB)

 Trainable params: 4,655,523 (17.76 MB)

 Non-trainable params: 1,841,984 (7.03 MB)

In [85]:
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]


def show_predictions(epoch, dataset=None, num=50):
  if dataset:

    for image, mask in dataset.take(num):
        pred_mask = model.predict(image)
        plt.figure(figsize=(15, 10))
        plt.subplot(231)
        plt.title('Testing Image')
        plt.imshow(image[0], cmap='gray')
        plt.subplot(232)
        plt.title('Ground Truth')
        plt.imshow(mask[0], cmap='jet')
        plt.subplot(233)
        plt.title('Prediction on test image')
        plt.imshow(create_mask(pred_mask), cmap='jet')

        plt.savefig(f"results/mask_{str(ii)}.png")

        plt.show()
  else:
      fig = plt.figure(figsize=(12, 12))
      fig.suptitle(f"\n Epoch: {str(epoch)}\n", fontsize=16)

      plt.subplot(331)
      plt.title('Testing Image')
      plt.imshow(train_images[num], cmap='gray')
      plt.subplot(332)
      plt.title('Ground Truth')
      plt.imshow(train_masks[num], cmap='jet')
      plt.subplot(333)
      plt.title('Prediction on test image')
      plt.imshow(create_mask(model.predict(train_images[num][tf.newaxis, ...]))[:,:,0], cmap='jet')

      plt.subplot(334)
      plt.imshow(train_images[num+16], cmap='gray')
      plt.subplot(335)
      plt.imshow(train_masks[num+16], cmap='jet')
      plt.subplot(336)
      plt.imshow(create_mask(model.predict(train_images[num+16][tf.newaxis, ...]))[:,:,0], cmap='jet')
      plt.subplot(337)
      plt.imshow(train_images[num+14], cmap='gray')
      plt.subplot(338)
      plt.imshow(train_masks[num+14], cmap='jet')
      plt.subplot(339)
      plt.imshow(create_mask(model.predict(train_images[num+14][tf.newaxis, ...]))[:,:,0], cmap='jet')

      #plt.savefig(f"results/mask_{str(num+100)}_{str(epoch)}.png")

      plt.show()

In [86]:
model = unet_model(OUTPUT_CHANNELS)
model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])   #


<ipython-input-84-af321efba355>:6: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=[IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS], include_top=False)


NameError: name 'pix2pix' is not defined

In [78]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(32, 256, 256, 3), output.shape=(32, 32, 32, 3)